In [1]:
import os

In [2]:
%pwd

'/home/vibhav911/Documents/skills/Data Science/DS_Projects/WineQuality/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vibhav911/Documents/skills/Data Science/DS_Projects/WineQuality'

In [5]:
# Environment variables
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/Vibhav911/WineQuality.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Vibhav911'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'da4547942d0e21905b789c56015bd7f201f66d4e'

In [20]:
# Entity program
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path : Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [21]:
from WineQProject.constants import *
from WineQProject.utils.common import read_yaml, create_directories

In [22]:
# Configuration Manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = 'https://dagshub.com/Vibhav911/WineQuality.mlflow'
        )
        
        return model_evaluation_config

In [23]:
# Component
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from WineQProject.utils.common import save_json

In [24]:
class ModelEvaluation:
    def __init__(self, config=ModelEvaluationConfig):
        self.config = config
        
        
    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)    # settting tracking uri as doing in the remote server
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x.values)
            (rmse, mae, r2) = self.eval_metric(test_y.values, predicted_qualities)
            
            #Saving metric as local
            scores={'rmse':rmse, 'mae':mae, 'r2':r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)    # saving in mlflow
            mlflow.log_metric('rmse', rmse)
            mlflow.log_metric('r2', r2)
            mlflow.log_metric('mae', mae)
            
            
            # Check if you are working in uri or local file
            # model registry does not work with file store
            if tracking_uri_type_store != 'file':
                
                # Register the model
                # There are other way to use the Model Registry, which depends on the use case, 
                # please refer to the document for more information:
                # https://www.mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, 'model', registered_model_name="ElasticnetModel")
            
            else:
                mlflow.sklearn.log_model(model,'model')
                
                # Save the model to the model registry
            

In [25]:
# pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-05-20 06:40:11,175: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-20 06:40:11,177: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-20 06:40:11,180: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-20 06:40:11,181: INFO: common: created directory at: artifacts]
[2024-05-20 06:40:11,182: INFO: common: created directory at: artifacts/model_evaluation]
[2024-05-20 06:40:11,206: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Vibhav911/WineQuality.mlflow/api/2.0/mlflow/runs/create]
[2024-05-20 06:40:11,822: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/home/vibhav911/Documents/skills/Data Science/DS_Projects/WineQuality/winequality_envp03/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/vibhav911/Documents/skills/Data Science/DS_Projects/WineQuality/winequality_envp03/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/05/20 06:40:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 se